## Backtesting result of 007 pytorch model

In [1]:
from backtesting import Backtest, Strategy
import pandas as pd

hist = pd.read_csv("hist5.csv")
# print(hist.shape)

df = pd.read_csv("df007.csv")

df['u_sig'] = hist['0'] > 0.3
df['d_sig'] = hist['0'] < 0

# set index
df['Datetime'] = pd.DatetimeIndex(df['Datetime'])
df = df.set_index('Datetime')

df

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

,Open,High,Low,Close,Adj Close,Volume,atr,u_sig,d_sig
Datetime,,,,,,,,,
2023-12-05 13:00:00-05:00,4565.129883,4565.790039,4562.390137,4563.959961,4563.959961,35681144,4.207037,False,True
2023-12-05 13:05:00-05:00,4563.939941,4565.330078,4561.660156,4563.620117,4563.620117,38296882,4.168672,False,True
2023-12-05 13:10:00-05:00,4563.720215,4564.930176,4562.950195,4564.379883,4564.379883,33246920,4.012337,False,True
2023-12-05 13:15:00-05:00,4564.390137,4566.930176,4564.120117,4566.569824,4566.569824,33388510,3.926460,False,True
2023-12-05 13:20:00-05:00,4566.649902,4568.399902,4566.299805,4567.970215,4567.970215,28071558,3.796005,False,False
...,...,...,...,...,...,...,...,...,...
2024-02-16 15:30:00-05:00,5016.120117,5017.200195,5013.790039,5016.020020,5016.020020,53007682,3.659394,False,False
2024-02-16 15:35:00-05:00,5016.140137,5016.500000,5011.629883,5012.060059,5012.060059,54369388,3.745874,True,False
2024-02-16 15:40:00-05:00,5012.089844,5012.089844,5005.359863,5006.509766,5006.509766,75685062,3.959025,False,False


In [2]:
class St(Strategy):
    def init(self):
        pass
        
    def next(self):
        if self.data['u_sig'][-1]: 
            self.buy()

        if self.position and self.data['d_sig'][-1]:
            self.position.close()

In [3]:
bt = Backtest(
    df,
    St,
    trade_on_close=True,
    exclusive_orders=True 
)

stats = bt.run()

stats

Start                     2023-12-05 13:00...
End                       2024-02-16 15:50...
Duration                     73 days 02:50:00
Exposure Time [%]                   41.219822
Equity Final [$]                 10913.408203
Equity Peak [$]                  10941.808594
Return [%]                           9.134082
Buy & Hold Return [%]                9.665507
Return (Ann.) [%]                   54.699259
Volatility (Ann.) [%]                8.194943
Sharpe Ratio                         6.674757
Sortino Ratio                       20.634319
Calmar Ratio                        40.029702
Max. Drawdown [%]                   -1.366467
Avg. Drawdown [%]                    -0.11422
Max. Drawdown Duration       11 days 18:25:00
Avg. Drawdown Duration        0 days 14:00:00
# Trades                                  778
Win Rate [%]                        58.997429
Best Trade [%]                       0.844991
Worst Trade [%]                     -0.466697
Avg. Trade [%]                    

In [4]:
bt.plot()

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='1408', ...)

Profit 109%，USD 913， 看起來好像還行，實際上錯的離譜，因為沒有算 commission，原本以為 0.002，千分之二的 commission 可以忽略不計，然而小數怕長計，SPX 一份 share USD 4500-5000， 也就是說每次買賣的 commission 已經來到 USD 10 上下，上面交易了778次，實際上付出了USD
 7千多的 commission..

In [5]:
bt = Backtest(
    df,
    St,
    commission=.002,
    trade_on_close=True,
    exclusive_orders=True 
)

stats = bt.run()

stats

Start                     2023-12-05 13:00...
End                       2024-02-16 15:50...
Duration                     73 days 02:50:00
Exposure Time [%]                   27.750953
Equity Final [$]                  4862.027807
Equity Peak [$]                       10000.0
Return [%]                         -51.379722
Buy & Hold Return [%]                9.665507
Return (Ann.) [%]                  -97.047543
Volatility (Ann.) [%]                0.604312
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -51.381123
Avg. Drawdown [%]                  -51.381123
Max. Drawdown Duration       73 days 00:50:00
Avg. Drawdown Duration       73 days 00:50:00
# Trades                                  529
Win Rate [%]                         1.890359
Best Trade [%]                       0.643704
Worst Trade [%]                     -0.665367
Avg. Trade [%]                    

In [6]:
bt.plot()

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='2098', ...)

虧到底褲都沒了